In [2]:
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Madrid

In [21]:
# Buscamos que la url sea como nosotros la pedimos

date_1 = '2024-10-25'
date_2 = '2024-10-27'
ciudad = 'viena'
pagina = 1

url = f"https://www.civitatis.com/es/{ciudad}/?page={pagina}&fromDate={date_1}&toDate={date_2}"

In [24]:
#Abrimos en navegador con la Url introducida
driver = webdriver.Chrome()
url_wunder = url

driver.get(url_wunder)
driver.maximize_window()
sleep(5)
driver.close()

In [25]:
# Comprobamos que la request sea corrrecta

url_civitatis = "https://www.civitatis.com/es/{ciudad}/?page={pagina}&fromDate={date_1}&toDate={date_2}"
res_civitatis = requests.get(url)
print(f"La respuesta de la petición es: {res_civitatis.status_code}")

La respuesta de la petición es: 200


In [26]:
#creamos nuestra sopa 

sopa_civitatis = BeautifulSoup(res_civitatis.content, "html.parser")

In [ ]:
# comprobamo snuestra sopa
sopa_civitatis

In [27]:
# Creamos nuestro bucle para iterar de los datos scrapeados y coger lo que no shace falta

nombres_actividades = []
descripciones_actividades = []
precios_actividades = []

bloque_total = sopa_civitatis.findAll("div", {"class": "o-search-list__item"} )
for i in bloque_total:
    nombre_actividad = i.find("h2").text.strip()
    descripcion_actividad = i.find("div",{"class":"comfort-card__text l-list-card__text"}).text.strip()
    precio_actividad = i.find("span",{"class":"comfort-card__price__text"}).text.strip()

# Lo añadimos a la lista de listas
    nombres_actividades.append(nombre_actividad)
    descripciones_actividades.append(descripcion_actividad)
    precios_actividades.append(precio_actividad)

# Creamos un documento
datos = {
    'Nombre Actividad': nombres_actividades,
    'Descripción Actividad': descripciones_actividades,
    'Precio Actividad': precios_actividades
}

# Creamos los datos de diccionario en un DataFrame
df_viena = pd.DataFrame(datos)
df_viena

,Nombre Actividad,Descripción Actividad,Precio Actividad
0,Concierto de Mozart en el Musikverein,¿Os gusta la música? Asistir a un concierto de...,59 €
1,Free tour por Viena,En este free tour por Viena recorreremos a pie...,¡Gratis!
2,Visita guiada por el Palacio Schönbrunn,En esta visita guiada por el Palacio Schönbrun...,54 €
3,Entrada al Museo Belvedere,Con esta entrada al Museo Belvedere visitaréis...,"14,60 €"
4,Tour de Sissi por el Palacio de Hofburg,Sigue las huellas de la legendaria emperatriz ...,48 €
5,Traslados en Viena,"Servicio privado, puerta a puerta y al mejor p...",60 €
6,Tour por Viena y Palacio Schönbrunn,Esta visita guiada por Viena es la forma ideal...,63 €
7,Excursión a Hallstatt,En esta excursión a Hallstatt desde Viena disf...,135 €
8,Excursión a Salzburgo,En esta excursión a Salzburgo desde Viena cono...,135 €
9,Tour de los misterios y leyendas de Viena,Descubrid cuáles han sido los episodios más cr...,21 €


In [28]:
#Guardamos el DF como CSV
df_viena.to_csv("../data/ActividadesViena.csv")